In [6]:
%run DensityPeakEstimation.ipynb

In [9]:
mat_data = scipy.io.loadmat('SalinasA.mat')
data = mat_data["salinasA"]
x, y, z = data.shape
salinasA = data.reshape((x*y, z))
norms = np.linalg.norm(salinasA, axis=1)
# Normalize each pixel vector by dividing by its norm
salinasA = salinasA / norms[:, np.newaxis]
mat_data_gt = scipy.io.loadmat('SalinasA_gt.mat')
gt = mat_data_gt["salinasA_gt"]
gt = gt.reshape(-1)

In [10]:
def land(X, K, GT, B, t):
    
    n = X.shape[0]
    Labels = np.zeros(n)
    
    Adj = make_adjacency(X, K)
    kernel = gaussian_kernel(Adj)
    diffusion = diffusion_map(kernel)
    
    
    Embed = embed(diffusion, t)
    embedding = Embed[0]
    PWDiffDist = diffusion_dist(embedding)
    PWdist = cdist(X, X)
    Delta_Density, Density_, Delta_ = find_density_peaks(X, K, PWdist, PWDiffDist)
    centers = density_peaks_estimation(Delta_Density, B)
    
    total_c = []
    
    t = B
    it = 0
    while t > 0:
        
        while GT[centers[it]] == 0:
            it += 1
        
            #print(centers[it])
        Labels[centers[it]] = GT[centers[it]] + 1 
        #print(centers[it])
        total_c.append(centers[it])
        t-=1
        it += 1
    print(np.where(Labels != 0))
    
    
    # Sort points by density
    siD = sorted(enumerate(Density_), key=lambda x: x[1], reverse=True)
    
    idx = np.array([int(index) for index, _ in siD])

    
    # Label highest density point if necessary
    
    if Labels[idx[0]] == 0:
        Labels[idx[0]] = GT[idx[0]] + 1
    
    
    # First pass
    row_max = np.max(diffusion, axis=1)

    for j in range(n):
        if Labels[idx[j]] == 0:
            NN = np.argmin(PWDiffDist[idx[j], idx[:j]])
            Labels[idx[j]] = Labels[idx[NN]]
    print(set(total_c))
    print(len(set(total_c)))
    return Labels

In [88]:
def lend(X, K, GT, B, t):
    
    n = X.shape[0]
    Labels = np.zeros(n)
    Adj = make_adjacency(X, K)
    kernel = gaussian_kernel(Adj)
    diffusion = diffusion_map(kernel)
    Embed = embed(diffusion, t)
    embedding = Embed[0]
    PWdist = cdist(X, X)
    PWDiffDist = diffusion_dist(embedding)
    Delta_Density, Density_, Delta_ = find_density_peaks(X, K, PWDiffDist, PWDiffDist)
    #centers = density_peaks_estimation(Delta_Density, B)
    #print(centers)
    total_c = []
    for iter in range(3):
        
        Labels = np.zeros(n)
        #print(Labels)
        #print(iter)
        Embed = embed(diffusion, t)
        embedding = Embed[0]
        PWDiffDist = diffusion_dist(embedding)
        
        Delta_Density, Density_, Delta_ = find_density_peaks(X, K, PWDiffDist, PWDiffDist, Density_)
        centers = density_peaks_estimation(Delta_Density, B - (2 - iter))
        #print(centers)
        #total_c.extend(centers)
        t = B - (2 - iter)
        it = 0
        while t > 0:
            while GT[centers[it]] == 0:
                it += 1
            Labels[centers[it]] = GT[centers[it]] + 1 #+1 to ensure unlabeled points have value of 0
            #print(centers[it])
            total_c.append(centers[it])
            t-=1
            it += 1
        '''
        for i in range(B - (2 - iter)):
            while GT[centers[i]] == 0:
                i += 1
            Labels[centers[i]] = GT[centers[i]] + 1 #+1 to ensure unlabeled points have value of 0
            print(centers[i])
            total_c.append(centers[i])
        '''
        # Sort points by density
        siD = sorted(enumerate(Density_), key=lambda x: x[1], reverse=True)
        # Extracting just the indices and converting to NumPy array
        idx = np.array([int(index) for index, _ in siD])
        #epsilon_neighbors = [points_within_epsilon(X, idx, 0.01) for idx in range(X.shape[0])]
        # Label highest density point if necessary
        if Labels[idx[0]] == 0:
            Labels[idx[0]] = GT[idx[0]] + 1
        # First pass
        row_max = np.max(diffusion, axis=1)
        for j in range(n):
            if Labels[idx[j]] == 0:
                NN = np.argmin(PWDiffDist[idx[j], idx[:j]])
                Labels[idx[j]] = Labels[idx[NN]]
        mask = Labels[:, None] == Labels  # Create a mask where labels are equal
        diffusion_u = diffusion* mask  # Element-wise multiplication to set non-equal label positions to 0
        diffusion = (diffusion_u + 3*diffusion) /4
        row_sums = diffusion.sum(axis=1)
        diffusion = diffusion / row_sums[:, np.newaxis]
        
    print(len(set(total_c)))
    print(set(total_c))
    return Labels
    